In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from deep_visual_odometry.kitti_utils import KITTIdata
from matplotlib import pyplot as plt

In [3]:
basedir = 'dataset'
kitti_data = KITTIdata(basedir, sequences = ['00','01'], img_size=(150,50))

completed load sequence 00 data
completed load sequence 01 data


In [4]:
inputs, velocities = kitti_data.load_data_input_model()

In [5]:
np.random.seed(100)
mask = np.arange(inputs.shape[0])
np.random.shuffle(mask)
inputs = inputs[mask]
velocities = velocities[mask]
num_validation = int(inputs.shape[0]/10)
num_training = inputs.shape[0] - num_validation

X_train, y_train = inputs[:num_training], velocities[:num_training]
X_val, y_val = inputs[num_training:], velocities[num_training:]

mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train /= 255
X_val /= 255

print(X_train.shape)
print(y_train.shape)
print(type(y_train[0,0]))
print(X_val.shape)
print(y_val.shape)

(5076, 50, 150, 6)
(5076, 2)
<class 'numpy.float64'>
(564, 50, 150, 6)
(564, 2)


In [8]:
from deep_visual_odometry.input_model import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, 
         conv_featmap=[16,16,16,16],
         fc_units=[128,128],
         conv_kernel_size=[7,5,5,5],
         pooling_size=[2,2,2,2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=20,
         batch_size=25,
         verbose=False,
         pre_trained_model=None)

Building velocity CNN. Parameters: 
conv_featmap=[16, 16, 16, 16]
fc_units=[128, 128]
conv_kernel_size=[7, 5, 5, 5]
pooling_size=[2, 2, 2, 2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 203
epoch 1 
Best validation mse! iteration:100 accuracy: 0.05740148201584816
epoch 2 
Best validation mse! iteration:300 accuracy: 0.052680667489767075
epoch 3 
Best validation mse! iteration:500 accuracy: 0.047352179884910583
epoch 4 
Best validation mse! iteration:700 accuracy: 0.04671459645032883
Best validation mse! iteration:800 accuracy: 0.04074319079518318
epoch 5 
Best validation mse! iteration:1000 accuracy: 0.04008011519908905
epoch 6 
epoch 7 
epoch 8 
epoch 9 
epoch 10 
epoch 11 
epoch 12 
epoch 13 
Best validation mse! iteration:2600 accuracy: 0.04006853327155113
epoch 14 
epoch 15 
epoch 16 
epoch 17 
Best validation mse! iteration:3300 accuracy: 0.036584869027137756
epoch 18 
epoch 19 
epoch 20 
Traning ends. The best valid accuracy is 0.036584869027137756. M

In [11]:
from deep_visual_odometry.input_model import test_input_model
print(X_val[0:2].shape)
out = test_input_model("CNN_Velocity_Model",X_val[0:2],y_val[0:2])
print(out)
print(y_val[0:2])

(2, 50, 150, 6)
INFO:tensorflow:Restoring parameters from model/CNN_Velocity_Model
[[ 1.2816603 -1.0440965]
 [ 0.9004614 -1.0440965]]
[[0.93924007 0.00172781]
 [0.5949605  0.0037615 ]]
